# Simple Control Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/control_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

[EPyT-Flow](https://github.com/WaterFutures/EPyT-Flow) is available on [PyPI](https://pypi.org/project/epyt-flow/) and can be installed via `pip install epyt-flow`:

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
from epyt.epanet import ToolkitConstants
from epyt_flow.data.networks import load_net1
from epyt_flow.simulation import ScenarioSimulator
from epyt_flow.simulation import SimpleControlModule
from epyt_flow.utils import to_seconds
from epyt_flow.simulation.events import ActuatorConstants

Create two simple control modules (i.e. entries in the "CONTROL" section of an .inp file) for controlling pump "9" based on the water leven in tank "2":

- LINK 9 OPEN IF NODE 2 BELOW 110
- LINK 9 CLOSED IF NODE 2 ABOVE 140

Note that all simple control modules must be instances of [SimpleControlModule](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.simple_control.SimpleControlModule).

In [ ]:
# LINK 9 OPEN IF NODE 2 BELOW 110
my_control_1 = SimpleControlModule(link_id="9",
                                   link_status=ActuatorConstants.EN_OPEN,
                                   cond_type=ToolkitConstants.EN_LOWLEVEL,
                                   cond_var_value="2",
                                   cond_comp_value=110.)

In [ ]:
# LINK 9 CLOSED IF NODE 2 ABOVE 140
my_control_2 = SimpleControlModule(link_id="9",
                                   link_status=ActuatorConstants.EN_CLOSED,
                                   cond_type=ToolkitConstants.EN_HILEVEL,
                                   cond_var_value="2",
                                   cond_comp_value=140.)

Create new simulation based on Net1:

In [ ]:
sim = ScenarioSimulator(scenario_config=load_net1(verbose=False))

Set simulation duration to two days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Monitor states of tank "2" and pump "9":

In [ ]:
sim.set_tank_sensors(sensor_locations=["2"])
sim.set_pump_state_sensors(sensor_locations=["9"])

Note that Net1.inp contains some simple controls. Remove all of them by calling [remove_all_simple_controls()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.remove_all_simple_controls):

In [ ]:
sim.remove_all_simple_controls()

Add our simple control modules by calling [add_simple_control()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_simple_control):

In [ ]:
sim.add_simple_control(my_control_1)
sim.add_simple_control(my_control_2)

Run the simulation and show sensor readings over time:

In [ ]:
scada_data = sim.run_simulation()

In [ ]:
scada_data.plot_pumps_state()

In [ ]:
scada_data.plot_tanks_water_volume()

Do not forget to close the simulation!

In [ ]:
sim.close()